In [1]:
# Import packages
import sqlite3
import pandas as pd
import numpy as np
import os

In [ ]:
# Config
DB_PATH = "cfb_data.db"
PRE_GAME_ELO_CSV_PATH = 'pre_game_elo.csv'

In [ ]:
# Data Consolidation

conn = sqlite3.connect(DB_PATH)

# Load games_full data - Select necessary columns
# Ensure you select scores, IDs, week/season, team info, neutral site, conf game,
# and ALL relevant spread/total columns (opening and closing)
games_query = """
SELECT
    g.id,
    g.season,
    g.week,
    g.season_type,
    g.completed,
    g.neutral_site,
    g.conference_game,
    g.attendance,
    g.home_team,
    g.home_conference,
    g.home_division,
    g.home_points,
    g.away_team,
    g.away_conference,
    g.away_division,
    g.away_points,
    g.avg_closing_spread,
    g.avg_closing_total,
    g.avg_opening_spread,
    g.avg_opening_total
    -- Add any other raw stats columns you might consider later
FROM
    games_full g
WHERE
    g.completed = 1 -- Only use completed games
ORDER BY
    g.season, g.week, g.id;
"""
print("Loading games data...")
games_df = pd.read_sql_query(games_query, conn)
print(f"Loaded {len(games_df)} completed games.")

# Close the database connection
conn.close()
print("Database connection closed.")